In [1]:
import pandas as pd
import numpy as np

In [449]:
df_main = pd.read_csv('table\data for matches\DATA premium teams, time, score, win 7.01+ to 13.02.18.csv', index_col=0)

In [450]:
df_main.head(5)

,match_id,start_time,radiant_team_id,radiant_score,dire_team_id,dire_score,radiant_win
0,2897064589,1483677730,1883502.0,23,350190.0,12,True
1,2897074199,1483678249,3214108.0,8,1375614.0,37,False
2,2897155244,1483682366,2581813.0,16,2512249.0,26,False
3,2897190211,1483683976,1836806.0,24,2659468.0,26,True
4,2897294037,1483688368,1883502.0,16,1375614.0,35,False


In [2]:
df_701_705 = pd.read_csv('table\data for matches\DATA premium hero_id, player_id, league_name 7.01 to 7.05.csv')
df_706_707 = pd.read_csv('table\data for matches\DATA premium hero_id, player_id, league_name 7.06 to 7.07.csv')
df_708 = pd.read_csv('table\data for matches\DATA premium hero_id, player_id, league_name 7.08 to 13.02.18.csv')

In [480]:
# Главная таблица с правильными столбцами
main = df_main#[df_main['match_id'] == id_match]
# присвоить ДФ для обработки из нужного файла с героями и игроками
df_her_play = df_708
# собрать все матчи из ДФ с героями и игроками
matches = np.unique(df_her_play['match_id'])
for id_match in matches:
    # номер индекса в галвном ДФ
    main_index = main[main['match_id'] == id_match].index[0]
    # создать два датафрейма по героям и игрокам (разделенных на выйгрывших и проигравших)
    her_false = pd.DataFrame(df_her_play['hero_id'][df_her_play['match_id'] == id_match][df_her_play['win'] == False])
    her_true = pd.DataFrame(df_her_play['hero_id'][df_her_play['match_id'] == id_match][df_her_play['win'] == True])
    heroes = pd.concat([her_false, her_true]).reset_index().T.drop('index')
    play_false = pd.DataFrame(df_her_play['account_id'][df_her_play['match_id'] == id_match][df_her_play['win'] == False])
    play_true = pd.DataFrame(df_her_play['account_id'][df_her_play['match_id'] == id_match][df_her_play['win'] == True])
    players = pd.concat([play_false, play_true]).reset_index().T.drop('index')

    # соединение и присваивание нормальных имен столбцам в дф с героями и игроками
    col_her = {0: 'lose_H1', 1: 'lose_H2', 2: 'lose_H3', 3: 'lose_H4', 4: 'lose_H5', 
                 5: 'win_H1', 6: 'win_H2', 7: 'win_H3', 8: 'win_H4', 9: 'win_H5'}
    col_play = {0: 'lose_P1', 1: 'lose_P2', 2: 'lose_P3', 3: 'lose_P4', 4: 'lose_P5', 
                 5: 'win_P1', 6: 'win_P2', 7: 'win_P3', 8: 'win_P4', 9: 'win_P5'}
    heroes = heroes.rename(columns=col_her,).rename( {'hero_id': 0})
    players = players.rename(columns=col_play).rename( {'account_id': 0})

    match = pd.merge(heroes, players,  left_index=True, right_index=True)
    match['match_id'] = id_match
    
    #ЗАменить название столбцов win or lose на радиант и даер
    if main[main['match_id'] == id_match]['radiant_win'].bool() == False:
        for i in range(1,6):
            main.loc[main_index, 'radiant_H' + str(i)] = match['lose_H' + str(i)][0]
        for i in range(1,6):
            main.loc[main_index, 'dire_H' + str(i)] = match['win_H' + str(i)][0]
        for i in range(1,6):
            main.loc[main_index, 'radiant_P' + str(i)] = match['lose_P' + str(i)][0]
        for i in range(1,6):
            main.loc[main_index, 'dire_P' + str(i)] = match['win_P' + str(i)][0]

    elif main[main['match_id'] == id_match]['radiant_win'].bool() == True:
        for i in range(1,6):
            main.loc[main_index, 'radiant_H' + str(i)] = match['win_H' + str(i)][0]
        for i in range(1,6):
            main.loc[main_index, 'dire_H' + str(i)] = match['lose_H' + str(i)][0]
        for i in range(1,6):
            main.loc[main_index, 'radiant_P' + str(i)] = match['win_P' + str(i)][0]
        for i in range(1,6):
            main.loc[main_index, 'dire_P' + str(i)] = match['lose_P' + str(i)][0]

In [506]:
# присвоить ДФ для обработки из нужного файла с героями и игроками
df_her_play = df_701_705
# собрать все матчи из ДФ с героями и игроками
matches = np.unique(df_her_play['match_id'])
# добавить лигу
for id_match in matches:
    # номер индекса в галвном ДФ
    main_index = main[main['match_id'] == id_match].index[0]
    df_her_play_index = df_her_play[df_her_play['match_id'] == id_match].index[0]
    
    main.loc[main_index, 'league_name'] = df_her_play.loc[df_her_play_index,'leaguename']

In [507]:
main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3901 entries, 0 to 3900
Data columns (total 28 columns):
match_id           3901 non-null int64
start_time         3901 non-null int64
radiant_team_id    3781 non-null float64
radiant_score      3901 non-null int64
dire_team_id       3782 non-null float64
dire_score         3901 non-null int64
radiant_win        3901 non-null bool
radiant_H1         3901 non-null float64
radiant_H2         3901 non-null float64
radiant_H3         3901 non-null float64
radiant_H4         3901 non-null float64
radiant_H5         3901 non-null float64
dire_H1            3901 non-null float64
dire_H2            3901 non-null float64
dire_H3            3901 non-null float64
dire_H4            3901 non-null float64
dire_H5            3901 non-null float64
radiant_P1         3901 non-null float64
radiant_P2         3901 non-null float64
radiant_P3         3901 non-null float64
radiant_P4         3901 non-null float64
radiant_P5         3901 non-null float64
di

In [5]:
main = pd.read_csv('table\MAIN TABLE PREMIUM.csv', index_col=0)

In [6]:
main

,match_id,start_time,radiant_team_id,radiant_score,dire_team_id,dire_score,radiant_win,radiant_H1,radiant_H2,radiant_H3,...,radiant_P2,radiant_P3,radiant_P4,radiant_P5,dire_P1,dire_P2,dire_P3,dire_P4,dire_P5,league_name
0,2897064589,1483677730,1883502.0,23,350190.0,12,True,86.0,8.0,76.0,...,132851371.0,106573901.0,92423451.0,106809101.0,89871557.0,181716137.0,112377459.0,121052479.0,93616251.0,ESL One Genting 2017
1,2897074199,1483678249,3214108.0,8,1375614.0,37,False,21.0,55.0,71.0,...,86726887.0,87196890.0,40547474.0,47434686.0,139876032.0,117281554.0,82327674.0,149486894.0,87012746.0,ESL One Genting 2017
2,2897155244,1483682366,2581813.0,16,2512249.0,26,False,79.0,63.0,28.0,...,180012313.0,116782914.0,152545459.0,102099826.0,87382579.0,103735745.0,38628747.0,86700461.0,86725175.0,ESL One Genting 2017
3,2897190211,1483683976,1836806.0,24,2659468.0,26,True,20.0,11.0,28.0,...,101586543.0,118134220.0,102644565.0,111114687.0,119576842.0,126417273.0,94296097.0,101525357.0,113457795.0,ESL One Genting 2017
4,2897294037,1483688368,1883502.0,16,1375614.0,35,False,87.0,48.0,8.0,...,132851371.0,106573901.0,92423451.0,106809101.0,139876032.0,149486894.0,117281554.0,82327674.0,87012746.0,ESL One Genting 2017
5,2897415426,1483692961,1883502.0,20,1375614.0,9,True,87.0,8.0,11.0,...,132851371.0,106573901.0,92423451.0,106809101.0,139876032.0,149486894.0,117281554.0,82327674.0,87012746.0,ESL One Genting 2017
6,2897508853,1483696241,1883502.0,2,1375614.0,34,False,102.0,8.0,11.0,...,132851371.0,106573901.0,92423451.0,106809101.0,117281554.0,82327674.0,149486894.0,139876032.0,87012746.0,ESL One Genting 2017
7,2897669822,1483701484,2512249.0,18,1836806.0,45,False,28.0,50.0,55.0,...,103735745.0,38628747.0,86700461.0,86725175.0,129585121.0,101586543.0,118134220.0,102644565.0,111114687.0,ESL One Genting 2017
8,2897839994,1483706228,2512249.0,47,1836806.0,46,True,52.0,20.0,89.0,...,103735745.0,86725175.0,86700461.0,38628747.0,129585121.0,101586543.0,118134220.0,102644565.0,111114687.0,ESL One Genting 2017
9,2898057696,1483711679,2512249.0,42,1836806.0,14,True,14.0,86.0,42.0,...,103735745.0,86725175.0,86700461.0,38628747.0,129585121.0,101586543.0,118134220.0,102644565.0,111114687.0,ESL One Genting 2017
